In [1]:
import cv2 as cv
import sys
sys.path.append("..")
import slam_utils as su
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
img_1 = cv.imread("./1.png")
img_2 = cv.imread("./2.png")
keypoints_1, keypoints_2, matches = su.find_feature_matches(img_1, img_2)
print(f"Number of matches: {len(matches)}")

Max dist: 94.0
Min dist: 4.0
Number of matches: 79


In [3]:
R, t = su.pose_estimation_2d2d(keypoints_1, keypoints_2, matches)


fundamental_matrix is 
[[ 4.54443750e-06  1.33385558e-04 -1.79849925e-02]
 [-1.27565701e-04  2.26679480e-05 -1.41667843e-02]
 [ 1.81499464e-02  4.14605587e-03  1.00000000e+00]]
essential_matrix is 
[[-0.00216635  0.10710349  0.09822344]
 [-0.05307032  0.03077166 -0.69811902]
 [-0.05867768  0.69562353  0.02018946]]
homography_matrix is 
[[ 9.48831932e-01 -1.51560188e-01  3.37818532e+01]
 [ 4.01832734e-02  9.68309989e-01  7.00140362e+00]
 [-3.02186330e-05  5.07624222e-05  1.00000000e+00]]
R is [[ 0.99530005 -0.05374526  0.08055587]
 [ 0.05063772  0.99791026  0.04013642]
 [-0.08254468 -0.03586862  0.99594167]]
t is [[-0.97864836]
 [-0.13316516]
 [ 0.15657086]]


In [4]:
# Check E = t^R*scale
t_x = np.cross(np.eye(3), t.transpose())
print(f"t^R= \n{t_x*R}")  # I guess this is scale simular to E

t^R= 
[[ 0.          0.00841494 -0.01072724]
 [ 0.00792839 -0.          0.03927944]
 [-0.01099208  0.03510276  0.        ]]


In [5]:
# Check epipolar constraints (commented out)
# Convert pixel coordinates to camera coordinates
K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
# These have to be 2xN numpy arrays
pts_1 = np.empty((2, 0))
pts_2 = np.empty((2,0))
for m in matches:
    pt1 = su.pixel2cam(keypoints_1[m.queryIdx].pt, K)
    # y1 = np.array([[pt1[0]], [pt1[1]], [1]])
    pt2 = su.pixel2cam(keypoints_2[m.trainIdx].pt, K)
    # y2 = np.array([[pt2[0]], [pt2[1]], [1]])
    # d = np.dot(np.dot(y2.transpose(), t_x), np.dot(R, y1))
    # print(f"epipolar constraint: {d}")
    pts_1 = np.hstack([pts_1, np.array(pt1).reshape(-1, 1)])
    pts_2 = np.hstack([pts_2, np.array(pt2).reshape(-1, 1)])
points3D = su.triangulation(R, t, pts_1, pts_2)

In [6]:
def get_color(depth):
    up_th = 11.7
    low_th = 6.4
    th_range = up_th - low_th
    if depth > up_th:
        depth = up_th
    if depth < low_th:
        depth = low_th
    scaled_range = depth-low_th
    color = tuple(int(x) for x in (255 * scaled_range/th_range, 0, 255 * (1 - scaled_range/th_range)))
    return color


# plot the points with color depth
img1_plot = img_1.copy()
img2_plot = img_2.copy()
for i in range(len(matches)):
    depth1 = points3D[i][2]
    pix1 = tuple(int(x) for x in keypoints_1[matches[i].queryIdx].pt)
    pt1_cam = su.pixel2cam(keypoints_1[matches[i].queryIdx].pt, K)
    cv.circle(img1_plot, pix1, 2, get_color(depth1), 2)
    pt2_trans = R.dot(points3D[i]) + t
    depth2 = pt2_trans[2]
    pix2 = tuple(int(x) for x in keypoints_2[matches[i].trainIdx].pt)
    cv.circle(img2_plot, pix2, 2, get_color(depth2), 2)
cv.imshow("img_1", img1_plot)
cv.imshow("img_2", img2_plot)
cv.waitKey()

-1